# Algorithme EM

In [2]:
%matplotlib inline
import matplotlib
matplotlib.use('TkAgg')
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

c:\users\margot\miniconda3\lib\site-packages\matplotlib\__init__.py:1350: UserWarning:  This call to matplotlib.use() has no effect
because the backend has already been chosen;
matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

  warnings.warn(_use_error_msg)


In [3]:
wine = pd.read_csv('Datasets/Debug.csv')

In [4]:
d = 11  #descripteurs
N =  wine.shape[0] #produits à tester
R = 5 #experts

y = np.random.rand(N,R)
x = np.random.rand(N,d)
w0 = np.random.rand(1,d)

## Initialisation

In [19]:
def init_mu(y):
	mu = []
	for i in range(0,N):
		mu.append(np.sum(y[i])/R)
	return mu

mu0 = init_mu(y)

def sigma(z):
    return 1/(1+np.exp(-z))

## Maximum Likelihood Estimator

In [34]:
def likelihood(w,y,alpha,beta) :
    prod = 1
    for i in range(1,N):
        p = sigma(x[i].dot(w.T))
        proda = 1
        for j in range(1,R):
            proda = proda*alpha[j]**(y[i][j])*(1-alpha[j])**(1-y[i][j])
        prodb = 1
        for j in range(1,R):
            prodb = prodb*beta[j]**(y[i][j])*(1-beta[j])**(1-y[i][j])
        prod = prod * (proda*p+prodb*(1-p))
    print(prod)

In [35]:
likelihood(w0,y,alpha0,beta0)

[  6.87659936e-119]


## Étape E

In [6]:
#E-step

def ai(alpha,y):
    a = []
    for i in range(0,N):
        proda = 1
        for j in range(0,R):
            proda = proda*alpha[j]**(y[i][j])*(1-alpha[j])**(1-y[i][j])
        a.append(proda)
    return a

def bi(beta,y):
    b = []
    for i in range(0,N):
        prodb = 1
        for j in range(0,R):
            prodb = prodb*beta[j]**(1-y[i][j])*(1-beta[j])**(y[i][j])
        b.append(prodb)
    return b

def pi(x,w):
    p = []
    for i in range(0,N):
        p.append(sigma(x[i].dot(w.T)))
    return p

def mui(a,b,p):
    mu = []
    for i in range(0,N):
        mu.append(a[i]*p[i]/(a[i]*p[i]+b[i]*(1-p[i])))
    return mu

def E_step(x,y,alpha,beta,w):
    CE = 0 #Conditionnal excepectation
    a = ai(alpha,y)
    b = bi(beta,y)
    p = pi(x,w)
    mu = mui(a,b,p)
    for i in range(0,N):
        CE += mu[i]*np.log(p[i])*a[i]+(1-mu[i])*np.log(1-p[i])*b[i]
    return CE


## Étape M

In [13]:
#M-step

def alpha(mu,y):
	alpha = []
	for j in range(0,R):
		tmp1 = 0
		tmp2 = 0
		for i in range(0,N):
			tmp1 += mu[i]*y[i][j]
			tmp2 += mu[i]
		alpha.append(tmp1/tmp2)
	return alpha

def beta(mu,y):
	beta = []
	for j in range(0,R):
		tmp1 = 0
		tmp2 = 0
		for i in range(0,N):
			tmp1 += (1-mu[i])*(1-y[i][j])
			tmp2 += 1-mu[i]
		beta.append(tmp1/tmp2)
	return beta

def updateW(w,x, eta):
    g = 0
    for i in range(0,N):
        g += (mu0[i] - sigma(x[i].dot(w.T)))*x[i]
    
    H = np.zeros((d,d))
    for i in range(0,N):
        H -= sigma(x[i].dot(w.T))*(1-sigma(x[i].dot(w.T)))*((x[i].reshape(11,1))*(x[i].reshape(1,11)))
    w = w - eta*np.linalg.inv(H).dot(g)
    print(w)

In [14]:
updateW(w0,x,0.1)

[[ 0.57942095  0.12666681  0.21056922  0.46481199  0.41258936  0.39248252
   0.3187013   0.32482651  0.41495374  0.45715522  0.27577648]]


In [9]:
alpha0 = alpha(mu0,y)
beta0 = beta(mu0,y)
print(alpha0,beta0)

[0.54470834506124255, 0.51159957932484323, 0.50164498276825853, 0.56194998475131219, 0.5136291221387258] [0.51072900154095102, 0.55334716045786125, 0.5513728187108029, 0.49863783292651837, 0.55516739720209929]


In [10]:
#a0 = a(alpha0,y)
#b0 = a(beta0,y)
print(y[0][0])
E_step(x,y,alpha0,beta0,w0)

0.122316837936


array([-0.65324221])